# Policy Iteration Visualization with Pygame

This notebook demonstrates how **Policy Iteration** progressively improves the policy in a grid maze environment. We'll visualize:
- How the policy changes in each iteration
- How the value function evolves
- The convergence process toward the optimal policy

Each iteration will be displayed in real-time using Pygame, showing the transformation from a random initial policy to the optimal policy.

## 1. Import Required Libraries

Import all necessary libraries for environment creation, dynamic programming algorithms, and visualization.

In [752]:
!pip install gymnasium pygame numpy matplotlib
# ============================================================================
# SECTION 1: IMPORTS AND SETUP
# ============================================================================
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pygame
import time
from IPython.display import clear_output
import matplotlib.pyplot as plt
from matplotlib import cm
import sys

print("✓ All libraries imported successfully!")

✓ All libraries imported successfully!



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\abdul\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## 2. Grid Maze Environment With Proper Visualisation

We extend the original GridMazeEnv to support policy rendering with arrows, value function heatmaps, and iteration tracking.

In [753]:
# ============================================================================
# SECTION 2: ENHANCED GRID MAZE ENVIRONMENT
# ============================================================================

class GridMazeEnvWithVisualization(gym.Env):
    """
    Enhanced GridMaze environment with built-in policy and value visualization.
    
    Additions to original environment:
    - Policy rendering (arrows showing action directions)
    - Value function heatmap
    - Iteration counter display
    - Highlighting of changed states
    """
    
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 4}
    
    def __init__(self, render_mode=None, size=5):
        self.size = size
        self.window_size = 800  
        
        # Action and observation spaces
        self.action_space = spaces.Discrete(4)
        low = np.zeros(8, dtype=int)  # Agent(2) + Goal(2) + Bad1(2) + Bad2(2)
        high = np.full(8, self.size - 1, dtype=int)
        self.observation_space = spaces.Box(low=low, high=high, dtype=int)

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode
        self.window = None
        self.clock = None
        
        # Visualization attributes
        self.font = None
        self.small_font = None

    def _get_obs(self):
        return np.concatenate([
            self._agent_location,
            self._goal_location,
            self._bad_cells[0],
            self._bad_cells[1]
        ])
    
    def _get_info(self):
        return {}

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        all_cells = np.array(range(self.size * self.size))
        selected_indices = self.np_random.choice(all_cells, size=4, replace=False)
        
        def index_to_coords(index):
            return np.array([index % self.size, index // self.size])

        self._agent_location = index_to_coords(selected_indices[0])
        self._goal_location = index_to_coords(selected_indices[1])
        self._bad_cells = [
            index_to_coords(selected_indices[2]),
            index_to_coords(selected_indices[3])
        ]
        
        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info

    def step(self, action):
        # Stochastic transitions (same as before)
        outcome_actions = [
            (action - 1) % 4,
            action,
            (action + 1) % 4
        ]
        
        effective_move_index = self.np_random.choice([0, 1, 2], p=[0.15, 0.70, 0.15])
        effective_action = outcome_actions[effective_move_index]

        movement = {
            0: np.array([1, 0]),
            1: np.array([0, -1]),
            2: np.array([-1, 0]),
            3: np.array([0, 1])
        }[effective_action]
        
        new_location = np.clip(
            self._agent_location + movement, 0, self.size - 1
        )
        self._agent_location = new_location

        terminated = False
        reward = -1.0

        if np.array_equal(self._agent_location, self._goal_location):
            reward = 100.0
            terminated = True
        
        for bad_cell in self._bad_cells:
            if np.array_equal(self._agent_location, bad_cell):
                reward = -100.0
                terminated = True
                break

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, False, info

    def _render_frame(self):
        # Initialize pygame if not already done (needed for both human and rgb_array modes)
        if self.font is None:
            pygame.init()
            self.font = pygame.font.Font(None, 36)
            self.small_font = pygame.font.Font(None, 24)
        
        # Initialize window only for human mode
        if self.window is None and self.render_mode == "human":
            pygame.display.init()
            self.window = pygame.display.set_mode((self.window_size, self.window_size))
            pygame.display.set_caption("Grid Maze - Policy Iteration Visualization")
        
        if self.clock is None and self.render_mode == "human":
            self.clock = pygame.time.Clock()

        canvas = pygame.Surface((self.window_size, self.window_size))
        canvas.fill((255, 255, 255))
        pix_sq_size = self.window_size / self.size

        # Draw grid
        for x in range(self.size + 1):
            pygame.draw.line(
                canvas, (200, 200, 200), 
                (0, pix_sq_size * x), (self.window_size, pix_sq_size * x), width=3
            )
            pygame.draw.line(
                canvas, (200, 200, 200), 
                (pix_sq_size * x, 0), (pix_sq_size * x, self.window_size), width=3
            )

        # Draw goal
        pygame.draw.rect(
            canvas, (0, 255, 0), pygame.Rect(
                pix_sq_size * self._goal_location[0],
                pix_sq_size * self._goal_location[1],
                pix_sq_size, pix_sq_size,
            ),
        )

        # Draw bad cells
        for bad_cell in self._bad_cells:
             pygame.draw.rect(
                canvas, (255, 0, 0), pygame.Rect(
                    pix_sq_size * bad_cell[0],
                    pix_sq_size * bad_cell[1],
                    pix_sq_size, pix_sq_size,
                ),
            )
        
        # Draw agent
        pygame.draw.circle(
            canvas, (0, 0, 255), (
                (self._agent_location[0] + 0.5) * pix_sq_size,
                (self._agent_location[1] + 0.5) * pix_sq_size,
            ), pix_sq_size / 3,
        )

        if self.render_mode == "human":
            self.window.blit(canvas, canvas.get_rect())
            pygame.event.pump()
            pygame.display.update()
            self.clock.tick(self.metadata["render_fps"])
        
        return canvas

    def render(self):
        """Return the rendered frame as RGB array or display to screen."""
        canvas = self._render_frame()
        
        if self.render_mode == "rgb_array":
            # Convert pygame surface to numpy array
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
            )
        # For human mode, rendering is already done in _render_frame
        return None

    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()

print("GridMazeEnv created successfully!")

GridMazeEnv created successfully!


## 3. Policy Visualization Renderer

Create a specialized class for rendering policies with arrows, value function heatmaps, and change highlighting.

In [754]:
# ============================================================================
# SECTION 3: POLICY VISUALIZATION RENDERER
# ============================================================================

class PolicyVisualizer:
    """
    Renders policy and value function with rich visual feedback.
    
    Features:
    - Arrow rendering for policy directions
    - Value function heatmap
    - Highlighting of changed states
    - Iteration counter and statistics
    """
    
    def __init__(self, env, window_size=800):
        self.env = env
        self.window_size = window_size
        self.window = None
        self.clock = None
        self.font = None
        self.small_font = None
        self.tiny_font = None
        
        # Color maps for value function
        self.colormap = cm.get_cmap('RdYlGn')  # Red (low) -> Yellow -> Green (high)
        
    def initialize_pygame(self):
        """Initialize pygame window and fonts."""
        if self.window is None:
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode((self.window_size, self.window_size + 100))
            pygame.display.set_caption("Policy Iteration - Real-time Visualization")
            self.clock = pygame.time.Clock()
            self.font = pygame.font.Font(None, 48)
            self.small_font = pygame.font.Font(None, 32)
            self.tiny_font = pygame.font.Font(None, 20)
    
    def render_policy_and_values(self, policy, V, iteration=0, changed_states=None, 
                                  title="Policy Iteration", delta=None):
        """
        Render the current policy with arrows and value function as heatmap.
        
        Args:
            policy: Current policy (action for each state)
            V: Current value function
            iteration: Current iteration number
            changed_states: Set of state indices that changed
            title: Title to display
            delta: Convergence delta value
        """
        self.initialize_pygame()
        
        canvas = pygame.Surface((self.window_size, self.window_size + 100))
        canvas.fill((240, 240, 240))
        
        pix_sq_size = self.window_size / self.env.size
        
        # Normalize values for color mapping
        V_min, V_max = V.min(), V.max()
        if V_max - V_min > 0:
            V_normalized = (V - V_min) / (V_max - V_min)
        else:
            V_normalized = np.zeros_like(V)
        
        # Draw each cell
        for state_idx in range(self.env.size * self.env.size):
            x, y = state_idx % self.env.size, state_idx // self.env.size
            state_coords = np.array([x, y])
            
            # Determine cell color based on value function
            color_val = V_normalized[state_idx]
            rgb = self.colormap(color_val)[:3]  # Get RGB, ignore alpha
            cell_color = tuple(int(c * 255) for c in rgb)
            
            # Highlight changed states
            if changed_states and state_idx in changed_states:
                cell_color = (255, 255, 0)  # Yellow for changed
            
            # Check if terminal state
            is_goal = np.array_equal(state_coords, self.env._goal_location)
            is_bad = any(np.array_equal(state_coords, bad) for bad in self.env._bad_cells)
            
            # Draw cell background
            rect = pygame.Rect(
                pix_sq_size * x,
                pix_sq_size * y,
                pix_sq_size,
                pix_sq_size
            )
            
            if is_goal:
                pygame.draw.rect(canvas, (0, 200, 0), rect)  # Green for goal
            elif is_bad:
                pygame.draw.rect(canvas, (200, 0, 0), rect)  # Red for bad
            else:
                pygame.draw.rect(canvas, cell_color, rect)
            
            # Draw grid lines
            pygame.draw.rect(canvas, (100, 100, 100), rect, 2)
            
            # Draw value text
            if not (is_goal or is_bad):
                value_text = self.tiny_font.render(f"{V[state_idx]:.1f}", True, (0, 0, 0))
                text_rect = value_text.get_rect(
                    center=(x * pix_sq_size + pix_sq_size * 0.5, 
                           y * pix_sq_size + pix_sq_size * 0.2)
                )
                canvas.blit(value_text, text_rect)
            
            # Draw policy arrow (if not terminal)
            if not (is_goal or is_bad):
                action = policy[state_idx]
                self._draw_arrow(canvas, x, y, action, pix_sq_size)
        
        # Draw goal and bad cell labels
        goal_x, goal_y = self.env._goal_location
        goal_text = self.small_font.render("G", True, (255, 255, 255))
        canvas.blit(goal_text, (goal_x * pix_sq_size + pix_sq_size * 0.4,
                                goal_y * pix_sq_size + pix_sq_size * 0.4))
        
        for bad_cell in self.env._bad_cells:
            bad_x, bad_y = bad_cell
            bad_text = self.small_font.render("X", True, (255, 255, 255))
            canvas.blit(bad_text, (bad_x * pix_sq_size + pix_sq_size * 0.4,
                                   bad_y * pix_sq_size + pix_sq_size * 0.4))
        
        # Draw info panel at bottom
        info_y = self.window_size + 10
        title_text = self.font.render(f"{title} - Iteration {iteration}", True, (0, 0, 0))
        canvas.blit(title_text, (10, info_y))
        
        if delta is not None:
            delta_text = self.small_font.render(f"Delta: {delta:.6f}", True, (0, 0, 100))
            canvas.blit(delta_text, (10, info_y + 50))
        
        # Update display
        self.window.blit(canvas, canvas.get_rect())
        pygame.event.pump()
        pygame.display.update()
        self.clock.tick(2)  # 2 FPS for visualization
    
    def _draw_arrow(self, canvas, x, y, action, pix_sq_size):
        """Draw an arrow indicating the action direction."""
        center_x = x * pix_sq_size + pix_sq_size * 0.5
        center_y = y * pix_sq_size + pix_sq_size * 0.6
        
        arrow_length = pix_sq_size * 0.3
        arrow_width = 5
        
        # Arrow directions
        directions = {
            0: (arrow_length, 0),      # Right
            1: (0, -arrow_length),     # Up
            2: (-arrow_length, 0),     # Left
            3: (0, arrow_length)       # Down
        }
        
        dx, dy = directions[action]
        end_x = center_x + dx
        end_y = center_y + dy
        
        # Draw arrow line
        pygame.draw.line(canvas, (0, 0, 0), (center_x, center_y), (end_x, end_y), arrow_width)
        
        # Draw arrowhead
        if action == 0:  # Right
            points = [(end_x, end_y), (end_x - 10, end_y - 5), (end_x - 10, end_y + 5)]
        elif action == 1:  # Up
            points = [(end_x, end_y), (end_x - 5, end_y + 10), (end_x + 5, end_y + 10)]
        elif action == 2:  # Left
            points = [(end_x, end_y), (end_x + 10, end_y - 5), (end_x + 10, end_y + 5)]
        else:  # Down
            points = [(end_x, end_y), (end_x - 5, end_y - 10), (end_x + 5, end_y - 10)]
        
        pygame.draw.polygon(canvas, (0, 0, 0), points)
    
    def close(self):
        """Clean up pygame resources."""
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()

print("PolicyVisualizer created successfully!")

PolicyVisualizer created successfully!


## 4. Policy Evaluation with Visualization

Modified policy evaluation that updates the visualization during convergence.

In [755]:
# ============================================================================
# SECTION 4: POLICY EVALUATION WITH VISUALIZATION
# ============================================================================

def policy_evaluation_visual(env, policy, V, visualizer, iteration, gamma=0.9, theta=10):
    """
    Policy evaluation with real-time visualization of value function updates.
    
    Shows:
    - How values converge iteratively
    - Delta (maximum change) decreasing over time
    """
    eval_iteration = 0
    
    while True:
        delta = 0
        V_new = np.zeros_like(V)
        
        for state_idx in range(env.size * env.size):
            x, y = state_idx % env.size, state_idx // env.size
            state_coords = np.array([x, y])
            
            # Terminal states
            is_terminal = False
            if np.array_equal(state_coords, env._goal_location):
                V_new[state_idx] = 0
                is_terminal = True
            else:
                for bad_cell in env._bad_cells:
                    if np.array_equal(state_coords, bad_cell):
                        V_new[state_idx] = 0
                        is_terminal = True
                        break
            
            if is_terminal:
                continue
            
            # Bellman update
            action = policy[state_idx]
            v = 0
            
            movements = {
                0: np.array([1, 0]),
                1: np.array([0, -1]),
                2: np.array([-1, 0]),
                3: np.array([0, 1])
            }
            
            outcome_actions = [
                
                (action - 1) % 4,
                action,
                (action + 1) % 4
            ]
            
            for i, eff_action in enumerate(outcome_actions):
                prob = 0.70 if i == 1 else 0.15
                
                next_pos = np.clip(
                    state_coords + movements[eff_action], 
                    0, env.size - 1
                )
                next_state_idx = next_pos[0] + next_pos[1] * env.size
                
                r = -1.0
                if np.array_equal(next_pos, env._goal_location):
                    r = 100.0
                elif any(np.array_equal(next_pos, bad) for bad in env._bad_cells):
                    r = -100.0

                v += prob * (r + gamma * V[next_state_idx])
            
            V_new[state_idx] = v
            delta = max(delta, np.abs(V_new[state_idx] - V[state_idx]))
        
        V = V_new
        eval_iteration += 1
        
        # Visualize every few iterations
        if eval_iteration % 3 == 0 or delta < theta:
            visualizer.render_policy_and_values(
                policy, V, iteration, 
                title=f"Policy Evaluation (sweep {eval_iteration})",
                delta=delta
            )
        
        if delta < theta:
            break
    
    return V

print("policy_evaluation_visual function defined!")

policy_evaluation_visual function defined!


## 5. Policy Improvement with Visualization

Modified policy improvement that highlights which states changed their policy.

In [756]:
# ============================================================================
# SECTION 5: POLICY IMPROVEMENT WITH VISUALIZATION
# ============================================================================

def policy_improvement_visual(env, V, old_policy, visualizer, iteration, gamma=0.9):
    """
    Policy improvement with visualization of changed states.
    
    Highlights:
    - States where the policy action changed
    - Before/after comparison
    """
    policy = np.zeros(env.size * env.size, dtype=int)
    changed_states = set()
    
    for state_idx in range(env.size * env.size):
        x, y = state_idx % env.size, state_idx // env.size
        state_coords = np.array([x, y])
        
        # Terminal states
        if np.array_equal(state_coords, env._goal_location) or \
           any(np.array_equal(state_coords, bad) for bad in env._bad_cells):
            policy[state_idx] = 0
            continue

        # Compute Q-values
        q_values = np.zeros(env.action_space.n)
        
        movements = {
            0: np.array([1, 0]),
            1: np.array([0, -1]),
            2: np.array([-1, 0]),
            3: np.array([0, 1])
        }
        
        for action in range(env.action_space.n):
            q = 0
            
            outcome_actions = [
                (action - 1) % 4,
                action,
                (action + 1) % 4
            ]
            
            for i, eff_action in enumerate(outcome_actions):
                prob = 0.70 if i == 1 else 0.15
                
                next_pos = np.clip(
                    state_coords + movements[eff_action], 
                    0, env.size - 1
                )
                next_state_idx = next_pos[0] + next_pos[1] * env.size
                
                r = -1.0
                if np.array_equal(next_pos, env._goal_location):
                    r = 100.0
                elif any(np.array_equal(next_pos, bad) for bad in env._bad_cells):
                    r = -100.0

                q += prob * (r + gamma * V[next_state_idx])
            
            q_values[action] = q

        # Choose best action
        best_action = np.argmax(q_values)
        policy[state_idx] = best_action
        
        # Track if policy changed
        if best_action != old_policy[state_idx]:
            changed_states.add(state_idx)
    
    # Visualize the improved policy with highlighted changes
    visualizer.render_policy_and_values(
        policy, V, iteration, changed_states=changed_states,
        title=f"Policy Improvement ({len(changed_states)} states changed)"
    )
    time.sleep(1)  # Pause to see the changes
    
    return policy

print("policy_improvement_visual function defined!")

policy_improvement_visual function defined!


## 6. Policy Iteration with Real-time Visualization

Complete Policy Iteration algorithm with step-by-step pygame visualization showing the entire convergence process.

In [757]:
# ============================================================================
# SECTION 6: POLICY ITERATION WITH REAL-TIME VISUALIZATION
# ============================================================================

def policy_iteration_with_visualization(env, gamma=0.95, max_iterations=20):
    """
    Complete Policy Iteration with real-time pygame visualization.
    
    Shows:
    - Initial random policy
    - Each evaluation phase (value convergence)
    - Each improvement phase (policy changes)
    - Final optimal policy
    """
    print("=" * 70)
    print("POLICY ITERATION WITH REAL-TIME VISUALIZATION")
    print("=" * 70)
    print(f"\nEnvironment Configuration:")
    print(f"  Goal (Green G): {env._goal_location}")
    print(f"  Bad Cells (Red X): {env._bad_cells}")
    print(f"  Discount Factor γ: {gamma}")
    print(f"\nStarting visualization... Watch the pygame window!")
    print("=" * 70)
    
    # Initialize
    visualizer = PolicyVisualizer(env)
    policy = np.random.choice(env.action_space.n, size=env.size * env.size)
    V = np.zeros(env.size * env.size)
    
    # Show initial random policy
    print("\n[Iteration 0] Showing initial RANDOM policy...")
    visualizer.render_policy_and_values(
        policy, V, 0, 
        title="Initial Random Policy"
    )
    time.sleep(2)
    
    policy_stable = False
    iteration = 0
    
    # Main Policy Iteration loop
    while not policy_stable and iteration < max_iterations:
        iteration += 1
        print(f"\n{'='*70}")
        print(f"[Iteration {iteration}] POLICY EVALUATION phase...")
        print(f"{'='*70}")
        
        # STEP 1: Policy Evaluation (with visualization)
        V = policy_evaluation_visual(env, policy, V, visualizer, iteration, gamma)
        print(f"  ✓ Value function converged!")
        
        print(f"\n[Iteration {iteration}] POLICY IMPROVEMENT phase...")
        
        # STEP 2: Policy Improvement (with visualization)
        new_policy = policy_improvement_visual(env, V, policy, visualizer, iteration, gamma)
        
        # STEP 3: Check convergence
        if np.array_equal(new_policy, policy):
            policy_stable = True
            print(f"\n{'='*70}")
            print(f"  ✓✓✓ CONVERGENCE ACHIEVED! ✓✓✓")
            print(f"  Optimal policy found in {iteration} iterations!")
            print(f"{'='*70}")
            
            # Show final optimal policy
            visualizer.render_policy_and_values(
                new_policy, V, iteration, 
                title="★ OPTIMAL POLICY ★"
            )
        else:
            num_changes = np.sum(new_policy != policy)
            print(f"  → {num_changes} states changed, continuing...")
        
        policy = new_policy
    
    if not policy_stable:
        print(f"\n⚠ Reached maximum iterations ({max_iterations})")
    
    print("\n" + "="*70)
    print("Visualization complete! Close the pygame window to continue.")
    print("="*70)
    
    # Keep window open until user closes it
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        time.sleep(0.1)
    
    visualizer.close()
    
    return policy, V, iteration

print("✓ policy_iteration_with_visualization function defined!")

✓ policy_iteration_with_visualization function defined!


## 7. Interactive Policy Comparison with Custom Rewards

Configure custom rewards and see their effect on the entire policy iteration process.

In [758]:
# ============================================================================
# SECTION 7: REWARD CONFIGURATION FOR POLICY ITERATION
# ============================================================================

print("="*70)
print("REWARD CONFIGURATION")
print("="*70)

# Allow custom reward configuration
print("\n📝 Configure Rewards for Policy Iteration (press Enter for defaults):")
print("Current defaults: step=-1, goal=+100, bad=-100, gamma=0.99\n")

try:
    step_reward_input = input("Step reward (default -1): ").strip()
    STEP_REWARD = float(step_reward_input) if step_reward_input else -1.0
    
    goal_reward_input = input("Goal reward (default +100): ").strip()
    GOAL_REWARD = float(goal_reward_input) if goal_reward_input else 100.0
    
    bad_reward_input = input("Bad cell penalty (default -100): ").strip()
    BAD_REWARD = float(bad_reward_input) if bad_reward_input else -100.0
    
    gamma_input = input("Discount factor gamma (default 0.99): ").strip()
    GAMMA = float(gamma_input) if gamma_input else 0.99
except:
    print("Invalid input, using defaults")
    STEP_REWARD = -1.0
    GOAL_REWARD = 100.0
    BAD_REWARD = -100.0
    GAMMA = 0.99

print(f"\n✓ Rewards Configured:")
print(f"  Step Reward:      {STEP_REWARD:+.1f}")
print(f"  Goal Reward:      {GOAL_REWARD:+.1f}")
print(f"  Bad Cell Penalty: {BAD_REWARD:+.1f}")
print(f"  Discount Factor:  {GAMMA:.2f}")

REWARD CONFIGURATION

📝 Configure Rewards for Policy Iteration (press Enter for defaults):
Current defaults: step=-1, goal=+100, bad=-100, gamma=0.99


✓ Rewards Configured:
  Step Reward:      -1.0
  Goal Reward:      +100.0
  Bad Cell Penalty: -100.0
  Discount Factor:  0.99

✓ Rewards Configured:
  Step Reward:      -1.0
  Goal Reward:      +100.0
  Bad Cell Penalty: -100.0
  Discount Factor:  0.99


In [759]:
# ============================================================================
# CUSTOM POLICY EVALUATION WITH CONFIGURABLE REWARDS
# ============================================================================

def policy_evaluation_visual_custom(env, policy, V, visualizer, iteration, gamma=0.99, theta=10):
    """Policy evaluation with custom rewards from user configuration."""
    eval_iteration = 0
    
    while True:
        delta = 0
        V_new = np.zeros_like(V)
        
        for state_idx in range(env.size * env.size):
            x, y = state_idx % env.size, state_idx // env.size
            state_coords = np.array([x, y])
            
            # Terminal states
            is_terminal = False
            if np.array_equal(state_coords, env._goal_location):
                V_new[state_idx] = 0.0
                is_terminal = True
            else:
                for bad_cell in env._bad_cells:
                    if np.array_equal(state_coords, bad_cell):
                        V_new[state_idx] = 0.0
                        is_terminal = True
                        break
            
            if is_terminal:
                continue
            
            # Bellman update with custom rewards
            action = policy[state_idx]
            v = 0
            
            movements = {
                0: np.array([1, 0]),
                1: np.array([0, -1]),
                2: np.array([-1, 0]),
                3: np.array([0, 1])
            }
            
            outcome_actions = [
                (action - 1) % 4,
                action,
                (action + 1) % 4
            ]
            
            for i, eff_action in enumerate(outcome_actions):
                prob = 0.70 if i == 1 else 0.15
                
                next_pos = np.clip(
                    state_coords + movements[eff_action], 
                    0, env.size - 1
                )
                next_state_idx = next_pos[0] + next_pos[1] * env.size
                
                # Use custom rewards
                r = STEP_REWARD
                if np.array_equal(next_pos, env._goal_location):
                    r = GOAL_REWARD
                elif any(np.array_equal(next_pos, bad) for bad in env._bad_cells):
                    r = BAD_REWARD

                v += prob * (r + gamma * V[next_state_idx])
            
            V_new[state_idx] = v
            delta = max(delta, np.abs(V_new[state_idx] - V[state_idx]))
        
        V = V_new
        eval_iteration += 1
        
        # Visualize every few iterations
        if eval_iteration % 3 == 0 or delta < theta:
            visualizer.render_policy_and_values(
                policy, V, iteration, 
                title=f"Policy Evaluation (sweep {eval_iteration})",
                delta=delta
            )
        
        if delta < theta:
            break
    
    return V

# ============================================================================
# CUSTOM POLICY IMPROVEMENT WITH CONFIGURABLE REWARDS
# ============================================================================

def policy_improvement_visual_custom(env, V, old_policy, visualizer, iteration, gamma=0.99):
    """Policy improvement with custom rewards from user configuration."""
    policy = np.zeros(env.size * env.size, dtype=int)
    changed_states = set()
    
    for state_idx in range(env.size * env.size):
        x, y = state_idx % env.size, state_idx // env.size
        state_coords = np.array([x, y])
        
        # Terminal states
        if np.array_equal(state_coords, env._goal_location) or \
           any(np.array_equal(state_coords, bad) for bad in env._bad_cells):
            policy[state_idx] = 0
            continue

        # Compute Q-values with custom rewards
        q_values = np.zeros(env.action_space.n)
        
        movements = {
            0: np.array([1, 0]),
            1: np.array([0, -1]),
            2: np.array([-1, 0]),
            3: np.array([0, 1])
        }
        
        for action in range(env.action_space.n):
            q = 0
            
            outcome_actions = [
                (action - 1) % 4,
                action,
                (action + 1) % 4
            ]
            
            for i, eff_action in enumerate(outcome_actions):
                prob = 0.70 if i == 1 else 0.15
                
                next_pos = np.clip(
                    state_coords + movements[eff_action], 
                    0, env.size - 1
                )
                next_state_idx = next_pos[0] + next_pos[1] * env.size
                
                # Use custom rewards
                r = STEP_REWARD
                if np.array_equal(next_pos, env._goal_location):
                    r = GOAL_REWARD
                elif any(np.array_equal(next_pos, bad) for bad in env._bad_cells):
                    r = BAD_REWARD

                q += prob * (r + gamma * V[next_state_idx])
            
            q_values[action] = q

        # Choose best action
        best_action = np.argmax(q_values)
        policy[state_idx] = best_action
        
        # Track if policy changed
        if best_action != old_policy[state_idx]:
            changed_states.add(state_idx)
    
    # Visualize the improved policy with highlighted changes
    visualizer.render_policy_and_values(
        policy, V, iteration, changed_states=changed_states,
        title=f"Policy Improvement ({len(changed_states)} states changed)"
    )
    time.sleep(1)
    
    return policy

print("✓ Custom policy evaluation and improvement functions defined!")

✓ Custom policy evaluation and improvement functions defined!


In [ ]:
# ============================================================================
# RUN INTERACTIVE POLICY ITERATION WITH CUSTOM REWARDS
# ============================================================================

# Create environment and fix configuration for consistent DP
env = GridMazeEnvWithVisualization(render_mode=None, size=5)
obs, info = env.reset()  # Fixed seed for reproducibility

print("\n" + "="*70)
print("🎮 Starting Policy Iteration with Custom Rewards")
print("="*70)
print(f"\nUsing Rewards:")
print(f"  Step: {STEP_REWARD:+.1f}, Goal: {GOAL_REWARD:+.1f}, Bad: {BAD_REWARD:+.1f}, γ: {GAMMA:.2f}")
print("\nInstructions:")
print("  - Watch the pygame window to see policy evolution")
print("  - Arrows show the action for each state")
print("  - Colors show value function (Red=low, Green=high)")
print("  - Yellow highlights = states that changed in improvement")
print("  - Close the window when done to continue")
print("\nPress Enter to start...")
input()

# Initialize
visualizer = PolicyVisualizer(env)
policy = np.random.choice(env.action_space.n, size=env.size * env.size)
V = np.zeros(env.size * env.size)

# Show initial random policy
print("\n[Iteration 0] Showing initial RANDOM policy...")
visualizer.render_policy_and_values(
    policy, V, 0, 
    title="Initial Random Policy"
)
time.sleep(2)

policy_stable = False
iteration = 0
max_iterations = 20

# Main Policy Iteration loop
while not policy_stable and iteration < max_iterations:
    iteration += 1
    print(f"\n{'='*70}")
    print(f"[Iteration {iteration}] POLICY EVALUATION phase...")
    print(f"{'='*70}")
    
    # Policy Evaluation with custom rewards
    V = policy_evaluation_visual_custom(env, policy, V, visualizer, iteration, GAMMA)
    print(f"  ✓ Value function converged!")
    
    print(f"\n[Iteration {iteration}] POLICY IMPROVEMENT phase...")
    
    # Policy Improvement with custom rewards
    new_policy = policy_improvement_visual_custom(env, V, policy, visualizer, iteration, GAMMA)
    
    # Check convergence
    if np.array_equal(new_policy, policy):
        policy_stable = True
        print(f"\n{'='*70}")
        print(f"  ✓✓✓ CONVERGENCE ACHIEVED! ✓✓✓")
        print(f"  Optimal policy found in {iteration} iterations!")
        print(f"{'='*70}")
        
        # Show final optimal policy
        visualizer.render_policy_and_values(
            new_policy, V, iteration, 
            title="★ OPTIMAL POLICY ★"
        )
    else:
        num_changes = np.sum(new_policy != policy)
        print(f"  → {num_changes} states changed, continuing...")
    
    policy = new_policy

if not policy_stable:
    print(f"\n⚠ Reached maximum iterations ({max_iterations})")

print("\n" + "="*70)
print("Visualization complete! Close the pygame window to continue.")
print("="*70)

# Keep window open until user closes it
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    time.sleep(0.1)

visualizer.close()

# Store results
optimal_policy = policy
optimal_V = V
num_iterations = iteration

print(f"\n✓ Optimal policy computed in {num_iterations} iterations!")
print(f"\nFinal Value Function Statistics:")
print(f"  Maximum Value: {optimal_V.max():.2f}")
print(f"  Minimum Value: {optimal_V.min():.2f}")
print(f"  Mean Value: {optimal_V.mean():.2f}")



🎮 Starting Policy Iteration with Custom Rewards

Using Rewards:
  Step: -1.0, Goal: +100.0, Bad: -100.0, γ: 0.99

Instructions:
  - Watch the pygame window to see policy evolution
  - Arrows show the action for each state
  - Colors show value function (Red=low, Green=high)
  - Yellow highlights = states that changed in improvement
  - Close the window when done to continue

Press Enter to start...


C:\Users\abdul\AppData\Local\Temp\ipykernel_13580\3756624994.py:26: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  self.colormap = cm.get_cmap('RdYlGn')  # Red (low) -> Yellow -> Green (high)



[Iteration 0] Showing initial RANDOM policy...

[Iteration 1] POLICY EVALUATION phase...

[Iteration 1] POLICY EVALUATION phase...
  ✓ Value function converged!

[Iteration 1] POLICY IMPROVEMENT phase...
  ✓ Value function converged!

[Iteration 1] POLICY IMPROVEMENT phase...
  → 20 states changed, continuing...

[Iteration 2] POLICY EVALUATION phase...
  → 20 states changed, continuing...

[Iteration 2] POLICY EVALUATION phase...
  ✓ Value function converged!

[Iteration 2] POLICY IMPROVEMENT phase...
  ✓ Value function converged!

[Iteration 2] POLICY IMPROVEMENT phase...
  → 9 states changed, continuing...

[Iteration 3] POLICY EVALUATION phase...
  → 9 states changed, continuing...

[Iteration 3] POLICY EVALUATION phase...
  ✓ Value function converged!

[Iteration 3] POLICY IMPROVEMENT phase...
  ✓ Value function converged!

[Iteration 3] POLICY IMPROVEMENT phase...
  → 7 states changed, continuing...

[Iteration 4] POLICY EVALUATION phase...
  ✓ Value function converged!

[Iterat

## Testing the Optimal Policy with Custom Rewards

Now let's test the optimal policy learned with your custom rewards.

In [ ]:
# ============================================================================
# TESTING THE OPTIMAL POLICY WITH CUSTOM REWARDS
# ============================================================================

# Create a custom environment wrapper to use the configured rewards
class TestEnvWithCustomRewards(GridMazeEnvWithVisualization):
    def step(self, action):
        # Call parent step logic with custom rewards
        outcome_actions = [
            (action - 1) % 4,
            action,
            (action + 1) % 4
        ]
        
        effective_move_index = self.np_random.choice([0, 1, 2], p=[0.15, 0.70, 0.15])
        effective_action = outcome_actions[effective_move_index]

        movement = {
            0: np.array([1, 0]),
            1: np.array([0, -1]),
            2: np.array([-1, 0]),
            3: np.array([0, 1])
        }[effective_action]
        
        new_location = np.clip(
            self._agent_location + movement, 0, self.size - 1
        )
        self._agent_location = new_location

        terminated = False
        reward = STEP_REWARD  # Use custom step reward

        if np.array_equal(self._agent_location, self._goal_location):
            reward = GOAL_REWARD  # Use custom goal reward
            terminated = True
        
        for bad_cell in self._bad_cells:
            if np.array_equal(self._agent_location, bad_cell):
                reward = BAD_REWARD  # Use custom bad reward
                terminated = True
                break

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, False, info

print("\n" + "="*70)
print("TESTING OPTIMAL POLICY WITH CUSTOM REWARDS")
print("="*70)
print(f"\nTest Rewards:")
print(f"  Step: {STEP_REWARD:+.1f}, Goal: {GOAL_REWARD:+.1f}, Bad: {BAD_REWARD:+.1f}")

# Create test environment with visualization and custom rewards
test_env = TestEnvWithCustomRewards(render_mode="human", size=5)
obs, info = test_env.reset()

# Use same configuration as training
test_env._goal_location = env._goal_location
test_env._bad_cells = env._bad_cells
test_env._agent_location = np.array([4, 4])  # Start from corner

print(f"\nConfiguration:")
print(f"  Start: {test_env._agent_location}")
print(f"  Goal: {test_env._goal_location}")
print(f"  Bad Cells: {test_env._bad_cells}")
print(f"\nRunning optimal policy...")
print("(Note: May fail due to stochastic transitions!)\n")

action_names = ['Right →', 'Up ↑', 'Left ←', 'Down ↓']
total_reward = 0
max_steps = 50

# Run episode with optimal policy
for step in range(max_steps):
    x, y = test_env._agent_location
    state_idx = x + y * env.size
    
    action = optimal_policy[state_idx]
    print(f"Step {step+1:2d}: Pos=({x},{y}) → Action={action_names[action]}")
    
    obs, reward, terminated, truncated, info = test_env.step(action)
    total_reward += reward
    
    test_env._render_frame()
    time.sleep(0.5)
    
    if terminated or truncated:
        print(f"\n{'='*50}")
        if reward == 100.0:
            print("✓✓✓ SUCCESS! Reached the goal! ✓✓✓")
        elif reward == -100.0:
            print("✗ Hit a bad cell (due to stochastic transitions)")
        print(f"{'='*50}")
        time.sleep(2)
        break

test_env.close()

print(f"\nResults:")
print(f"  Total Reward: {total_reward:.1f}")
print(f"  Steps Taken: {step+1}")
print(f"  Outcome: {'SUCCESS' if total_reward > 0 else 'FAILED'}")


TESTING OPTIMAL POLICY WITH CUSTOM REWARDS

Test Rewards:
  Step: -100.0, Goal: +100.0, Bad: -100.0

Configuration:
  Start: [4 4]
  Goal: [2 3]
  Bad Cells: [array([0, 0]), array([2, 0])]

Running optimal policy...
(Note: May fail due to stochastic transitions!)

Step  1: Pos=(4,4) → Action=Left ←

Configuration:
  Start: [4 4]
  Goal: [2 3]
  Bad Cells: [array([0, 0]), array([2, 0])]

Running optimal policy...
(Note: May fail due to stochastic transitions!)

Step  1: Pos=(4,4) → Action=Left ←
Step  2: Pos=(3,4) → Action=Left ←
Step  2: Pos=(3,4) → Action=Left ←
Step  3: Pos=(3,4) → Action=Left ←
Step  3: Pos=(3,4) → Action=Left ←
Step  4: Pos=(2,4) → Action=Up ↑
Step  4: Pos=(2,4) → Action=Up ↑
Step  5: Pos=(3,4) → Action=Left ←
Step  5: Pos=(3,4) → Action=Left ←
Step  6: Pos=(2,4) → Action=Up ↑
Step  6: Pos=(2,4) → Action=Up ↑

✓✓✓ SUCCESS! Reached the goal! ✓✓✓

✓✓✓ SUCCESS! Reached the goal! ✓✓✓

Results:
  Total Reward: -400.0
  Steps Taken: 6
  Outcome: FAILED

Results:
  Total

## Summary

This notebook demonstrated:

1. **Policy Evaluation Visualization**: Watched values converge iteratively
2. **Policy Improvement Visualization**: Saw which states changed their actions
3. **Complete Policy Iteration**: Observed convergence to optimal policy over multiple iterations
4. **Real-time Rendering**: Used Pygame to show the entire DP process

Key observations:
- Policy typically converges in 3-5 iterations
- Value function converges within each evaluation phase
- Changed states (yellow) decrease with each iteration
- Final policy is deterministic and optimal for the known MDP

## 9. Record Agent Performance and Save Video

Now let's record the trained agent navigating the maze and save it as a video file. We'll capture frames and use OpenCV or imageio to create the video.

In [ ]:
# ============================================================================
# SECTION 9: VIDEO RECORDING OF AGENT PERFORMANCE
# ============================================================================

# Install imageio if not already installed
try:
    import imageio
except ImportError:
    !pip install imageio imageio-ffmpeg
    import imageio

print("✓ Video recording libraries ready!")
print("\nPreparing to record agent navigation...")

✓ Video recording libraries ready!

Preparing to record agent navigation...


In [ ]:
# Create environment in rgb_array mode for video recording
video_env = GridMazeEnvWithVisualization(render_mode="rgb_array", size=5)
obs, info = video_env.reset()

# Use the same maze configuration as before
video_env._goal_location = env._goal_location
video_env._bad_cells = env._bad_cells
video_env._agent_location = np.array([4, 4])  # Start from corner

print("Recording Configuration:")
print(f"  Start Position: {video_env._agent_location}")
print(f"  Goal Position: {video_env._goal_location}")
print(f"  Bad Cells: {video_env._bad_cells}")
print(f"  Video Output: maze_agent_recording.mp4")

# Record frames
frames = []
total_reward = 0
max_steps = 50

action_names = {0: "RIGHT", 1: "DOWN", 2: "LEFT", 3: "UP"}

print("\n" + "="*50)
print("RECORDING AGENT NAVIGATION")
print("="*50)

# Capture initial frame with agent at starting position
frame = video_env.render()
frames.append(frame)

for step in range(max_steps):
    # Get current state and action from optimal policy
    x, y = video_env._agent_location
    state_idx = x + y * video_env.size
    action = optimal_policy[state_idx]
    
    print(f"Step {step+1:2d}: Pos=({x},{y}) → Action={action_names[action]}")
    
    # Take action
    obs, reward, terminated, truncated, info = video_env.step(action)
    total_reward += reward
    
    # Capture frame AFTER taking the step
    frame = video_env.render()
    frames.append(frame)
    # Capture frame AFTER taking the step
    frame = video_env.render()
    frames.append(frame)
    
    # Check if episode ended
    if terminated or truncated:
        print(f"\n{'='*50}")
        if reward == 100.0:
            print("✓✓✓ SUCCESS! Reached the goal! ✓✓✓")
        elif reward == -100.0:
            print("✗ Hit a bad cell")
        print(f"{'='*50}")
        break

video_env.close()

print(f"\nRecording Complete!")
print(f"  Total Reward: {total_reward:.1f}")
print(f"  Steps Taken: {step+1}")
print(f"  Frames Captured: {len(frames)}")
print(f"  Outcome: {'SUCCESS' if total_reward > 0 else 'FAILED'}")

Recording Configuration:
  Start Position: [4 4]
  Goal Position: [2 3]
  Bad Cells: [array([0, 0]), array([2, 0])]
  Video Output: maze_agent_recording.mp4

RECORDING AGENT NAVIGATION
Step  1: Pos=(4,4) → Action=LEFT
Step  2: Pos=(4,3) → Action=LEFT
Step  3: Pos=(3,3) → Action=LEFT
Step  4: Pos=(3,4) → Action=LEFT
Step  5: Pos=(2,4) → Action=DOWN
Step  6: Pos=(1,4) → Action=RIGHT
Step  7: Pos=(2,4) → Action=DOWN

✓✓✓ SUCCESS! Reached the goal! ✓✓✓

Recording Complete!
  Total Reward: 94.0
  Steps Taken: 7
  Frames Captured: 15
  Outcome: SUCCESS
Step  6: Pos=(1,4) → Action=RIGHT
Step  7: Pos=(2,4) → Action=DOWN

✓✓✓ SUCCESS! Reached the goal! ✓✓✓

Recording Complete!
  Total Reward: 94.0
  Steps Taken: 7
  Frames Captured: 15
  Outcome: SUCCESS


In [ ]:
# Save frames as video using imageio
output_filename = "maze_agent_recording.mp4"

print(f"\nSaving video to: {output_filename}")
print("This may take a moment...")

# Save video at 2 FPS (2 frames per second) for better viewing
imageio.mimsave(
    output_filename, 
    frames, 
    fps=2,  # Slower playback for better visualization
    codec='libx264',
    quality=8
)

print(f"✓ Video saved successfully!")
print(f"✓ File: {output_filename}")
print(f"✓ Duration: ~{len(frames)/2:.1f} seconds")
print(f"\nYou can now view the video file showing the agent navigating the maze!")


Saving video to: maze_agent_recording.mp4
This may take a moment...
✓ Video saved successfully!
✓ File: maze_agent_recording.mp4
✓ Duration: ~7.5 seconds

You can now view the video file showing the agent navigating the maze!


### Optional: Display Video Preview

If you want to preview the video in the notebook, you can use the code below:

In [ ]:
# Display the video in the notebook (optional)
from IPython.display import Video, display
import os

if os.path.exists(output_filename):
    print(f"Displaying video: {output_filename}")
    display(Video(output_filename, width=600, embed=True))
else:
    print(f"Video file not found: {output_filename}")

Displaying video: maze_agent_recording.mp4


REWARD FUNCTION COMPARISON

📊 REWARD SCHEME COMPARISON:
--------------------------------------------------------------------------------

Current (Your Setup):
  Step Cost:        -1.0
  Goal Reward:     100.0
  Bad Penalty:    -100.0
  Discount (γ):     0.95
  Purpose: High rewards, moderate discount
  Effective Planning Horizon: ~20.0 steps
  Break-even Steps: ~100.0 (goal worth it if reached within this)

Option 1: Balanced (RECOMMENDED):
  Step Cost:        -1.0
  Goal Reward:      10.0
  Bad Penalty:     -10.0
  Discount (γ):     0.99
  Purpose: Balanced rewards, high discount - Best for learning
  Effective Planning Horizon: ~100.0 steps
  Break-even Steps: ~10.0 (goal worth it if reached within this)

Option 2: Time-Efficient:
  Step Cost:        -0.1
  Goal Reward:       1.0
  Bad Penalty:      -1.0
  Discount (γ):     0.95
  Purpose: Small penalties, encourages exploration
  Effective Planning Horizon: ~20.0 steps
  Break-even Steps: ~10.0 (goal worth it if reached within this